# Symbolical Calculations with Sympy

In [1]:
%load_ext autoreload
%autoreload 2

import os
ROOT_DIR = os.getcwd()[:os.getcwd().rfind('NVcenter')]+ 'NVcenter'
os.chdir(ROOT_DIR)

from NVcenter import *
from NVcenter.mitigation_sympy import *
plt.style.use('NVcenter-default')

import qutip as q
import numpy as np
import sympy as sp

In [2]:
# define symbols
T1, T2, omega = sp.symbols('T_1 T_2 omega', real=True, positive=True)
rho00, rho01, rho10, rho11 = sp.symbols('rho00 rho01 rho10 rho11', complex=True)
t = sp.Symbol('t', real=True, nonnegative=True)

## Pure Dephasing Channel

In [3]:
# define symbols
T2, omega = sp.symbols('T_2 omega', real=True, positive=True)
rho00, rho01, rho10, rho11 = sp.symbols('rho00 rho01 rho10 rho11', complex=True)
t = sp.Symbol('t', real=True, nonnegative=True)

# define system and dissipator
gamma = 1/(2*T2)
L_ops = [sp.sqrt(gamma) * Z]
H = 0.5 * omega * Z
rho = sp.Matrix([[rho00, rho01], [rho10, rho11]])

# contruct and invert map 
STM = construct_STM(rho, H, L_ops, t)

inv_STM = STM.inv()
inv_choi = STM_to_Choi(inv_STM)

# divide map for in CP maps 
choi_plus, choi_minus = get_CP_Choi(inv_choi)

# select p and D, this step should be done manually 
kraus_sum = choi_minus[0::2, 0::2] + choi_minus[1::2, 1::2]
sp.pprint(kraus_sum)
p = kraus_sum[1,1]
D_val = p - kraus_sum[0,0]
choi_D = sp.Matrix([[D_val,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])

# divide map for in CPTP maps 
choi_plus, choi_minus = get_CPTP_Choi(choi_plus, choi_minus, choi_D, p)

Choi plus is already TP, no D matrix is needed
Choi minus is already TP, no D matrix is needed
⎡     t                      ⎤
⎢     ──                     ⎥
⎢     T₂                     ⎥
⎢0.5⋅ℯ   - 0.5        0      ⎥
⎢                            ⎥
⎢                    t       ⎥
⎢                    ──      ⎥
⎢                    T₂      ⎥
⎣      0        0.5⋅ℯ   - 0.5⎦


In [4]:
print(is_extremal(choi_plus))
print(is_extremal(choi_minus))
print(is_ancilla_required(choi_plus))
print(is_ancilla_required(choi_minus))

Choi matrix already describes to an extremal map.
True
Choi matrix already describes to an extremal map.
True
If this return an identity the choi matrix describes a unitary and can be implemented without an ancilla. 
Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0], [0, 0, 0, 1.00000000000000]])
If this return an identity the choi matrix describes a unitary and can be implemented without an ancilla. 
Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])


In [32]:
print("Pure dephasing channel")
print("p value")
sp.pprint(p)
print("CPTP Choi plus")
sp.pprint(choi_plus)
print("CPTP Choi minus")
sp.pprint(choi_minus)

Pure dephasing channel
p value
     t       
     ──      
     T₂      
0.5⋅ℯ   - 0.5
CPTP Choi plus
⎡                        ⅈ⋅ω⋅t⎤
⎢    1.0      0  0  1.0⋅ℯ     ⎥
⎢                             ⎥
⎢     0       0  0      0     ⎥
⎢                             ⎥
⎢     0       0  0      0     ⎥
⎢                             ⎥
⎢     -ⅈ⋅ω⋅t                  ⎥
⎣1.0⋅ℯ        0  0     1.0    ⎦
CPTP Choi minus
⎡                  ⅈ⋅ω⋅t⎤
⎢   1      0  0  -ℯ     ⎥
⎢                       ⎥
⎢   0      0  0     0   ⎥
⎢                       ⎥
⎢   0      0  0     0   ⎥
⎢                       ⎥
⎢  -ⅈ⋅ω⋅t               ⎥
⎣-ℯ        0  0     1   ⎦


## Relaxation Channel

In [5]:
# define symbols
T1, omega = sp.symbols('T_1 omega', real=True, positive=True)
rho00, rho01, rho10, rho11 = sp.symbols('rho00 rho01 rho10 rho11', complex=True)
t = sp.Symbol('t', real=True, nonnegative=True)

# define system and dissipator
gamma = 1/(T1)
L_ops = [sp.sqrt(gamma) * sp.Matrix([[0,1], [0,0]])]
H = sp.Matrix([[0,0], [0,0]]) # 0.5 * omega * Z
rho = sp.Matrix([[rho00, rho01], [rho10, rho11]])

# contruct and invert map 
STM = construct_STM(rho, H, L_ops, t)

inv_STM = STM.inv()
inv_choi = STM_to_Choi(inv_STM)

# divide map for in CP maps 
choi_plus, choi_minus = get_CP_Choi(inv_choi)

# select p and D, this step should be done manually 
kraus_sum = choi_minus[0::2, 0::2] + choi_minus[1::2, 1::2]
sp.pprint(kraus_sum)
p = kraus_sum[1,1]
D_val = p - kraus_sum[0,0]
choi_D = sp.Matrix([[D_val,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])

# divide map for in CPTP maps 
choi_plus, choi_minus = get_CPTP_Choi(choi_plus, choi_minus, choi_D, p)

Choi plus is not TP, choose a D matrix
Choi minus is not TP, choose a D matrix
⎡0        0      ⎤
⎢                ⎥
⎢        t       ⎥
⎢        ──      ⎥
⎢        T₁      ⎥
⎣0  1.0⋅ℯ   - 1.0⎦


In [6]:
print(is_extremal(choi_plus))
print(is_extremal(choi_minus))
print(is_ancilla_required(choi_plus))
print(is_ancilla_required(choi_minus))

False
Choi matrix already describes to an extremal map.
True
If this return an identity the choi matrix describes a unitary and can be implemented without an ancilla. 
Matrix([[1.00000000000000, 0, 0, 0], [0, 1.0*exp(-t/T_1), 0, 0], [0, 0, 1.0*exp(-t/T_1), 0], [0, 0, 0, 1.00000000000000]])
If this return an identity the choi matrix describes a unitary and can be implemented without an ancilla. 
Matrix([[2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])


In [34]:
print("Relaxation channel")
print("p value")
sp.pprint(p)
print("CPTP Choi plus")
sp.pprint(choi_plus)
print("CPTP Choi minus")
sp.pprint(choi_minus)

Relaxation channel
p value
     t       
     ──      
     T₁      
1.0⋅ℯ   - 1.0
CPTP Choi plus
⎡                      -t  ⎤
⎢                      ────⎥
⎢                      2⋅T₁⎥
⎢   1.0     0  0  1.0⋅ℯ    ⎥
⎢                          ⎥
⎢    0      0  0      0    ⎥
⎢                          ⎥
⎢    0      0  0      0    ⎥
⎢                          ⎥
⎢     -t                   ⎥
⎢     ────                 ⎥
⎢     2⋅T₁                 ⎥
⎣1.0⋅ℯ      0  0     1.0   ⎦
CPTP Choi minus
⎡1  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  0⎦


## Combined Dephasing and Relaxation Channel

In [ ]:
# define symbols
T1, T2, T3, omega = sp.symbols('T_1 T_2 T_3 omega', real=True, positive=True)
gamma_relax = 1/(T1)
gamma_deph = 1/(2*T2)
rho00, rho01, rho10, rho11 = sp.symbols('rho00 rho01 rho10 rho11', complex=True)
rho = sp.Matrix([[rho00, rho01], [rho10, rho11]])
t = sp.Symbol('t', real=True, nonnegative=True)

# define system and dissipator
L_ops = [sp.sqrt(gamma_relax) * sp.Matrix([[0,1], [0,0]]),
        sp.sqrt(gamma_deph) * Z]
H = sp.Matrix([[0,0], [0,0]]) # 0.5 * omega * Z

# contruct and invert map 
STM = construct_STM(rho, H, L_ops, t)

inv_STM = sp.Matrix([[1,0,0,1-sp.exp(t/T1)], [0,sp.exp(t/T3),0,0], [0,0,sp.exp(t/T3),0], [0,0,0,sp.exp(t/T1)]])
inv_choi = STM_to_Choi(inv_STM)

# divide map for in CP maps 
choi_plus, choi_minus = get_CP_Choi(inv_choi)

# select p and D, this step should be done manually 
# kraus_sum = partial_trace(choi_minus)
# print(kraus_sum)
# p = kraus_sum[0,0]
# choi_D = sp.Matrix([[p,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])

# divide map for in CPTP maps 
# choi_plus, choi_minus = get_CPTP_Choi(choi_plus, choi_minus, choi_D, p)

In [63]:
# define symbols
T1, T2, T3, omega = sp.symbols('T_1 T_2 T_3 omega', real=True, positive=True)
gamma_relax = 1/(T1)
gamma_deph = 1/(2*T2)
rho00, rho01, rho10, rho11 = sp.symbols('rho00 rho01 rho10 rho11', complex=True)
rho = sp.Matrix([[rho00, rho01], [rho10, rho11]])
t = sp.Symbol('t', real=True, nonnegative=True)

# define system and dissipator
L_relax = sp.sqrt(gamma_relax) * sp.Matrix([[0,1], [0,0]])
L_deph = sp.sqrt(gamma_deph) * Z
H_0 = sp.Matrix([[0,0], [0,0]])
H_Z = 0.5 * omega * Z
H_X = 0.5 * omega * X

# contruct and invert map 
STM1 = sp.simplify(construct_STM(rho, H_Z, [L_relax, L_deph], t))

# divide map into three maps
STM_relax = construct_STM(rho, H_0, [L_relax], t)
STM_deph = construct_STM(rho, H_0, [L_deph], t)
STM_unitary = construct_STM(rho, H_Z, [], t)

STM2 = sp.simplify(STM_relax * STM_deph * STM_unitary)
sp.simplify(STM1 - STM2)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [33]:
# Calculate p * C_minus

C1 = Kraus_to_STM([Z])
C1 = STM_to_Choi(C1)

T1, T2 = sp.symbols('T_1 T_2', real=True, positive=True)
theta = sp.symbols('theta', real=True)
Rz = sp.Matrix([[sp.exp(-sp.I*theta/2), 0], [0, sp.exp(sp.I*theta/2)]])
C2 = Kraus_to_STM([Rz])
C2 = STM_to_Choi(C2)

C3 = sp.Matrix([[1,0,0,0], [0,0,0,0], [0,0,1,0], [0,0,0,0]])

p1 = 0.5 * sp.exp(t/T1) * 0.5 * ( sp.exp(t/T2) - 1)
p2 = p1
p3 = sp.exp(t/T1) - 1

## Extremal Maps and Circuit Decomposition

In [9]:
# is the second division needed or does the choi matrix already belong to an extremal map?
# for an extremal map we get C = sqrt(A) U sqrt(B)
# here I calculate constraints for the contraction matrix R 

def show_constraints(choi):
    A = choi[0:2, 0:2]
    B = choi[2:4, 2:4]
    C = choi[0:2, 2:4]
    C_dag = choi[2:4, 0:2]
    
    r00, r01, r10, r11 = sp.symbols('r00 r01 r10 r11', complex=True)
    R = sp.Matrix([[r00, r01], [r10, r11]])
    print("upper right")
    sp.pprint(C)
    sp.pprint( sp.sqrt(A) * R * sp.sqrt(B) )
    print("lower left")
    sp.pprint(C_dag)
    sp.pprint( sp.sqrt(B) * R.H * sp.sqrt(A) )

show_constraints(choi_plus)
show_constraints(choi_minus)

upper right
⎡        ⅈ⋅ω⋅t⎤
⎢0  1.0⋅ℯ     ⎥
⎢             ⎥
⎣0      0     ⎦
⎡0  1.0⋅r₀₁⎤
⎢          ⎥
⎣0     0   ⎦
lower left
⎡     0       0⎤
⎢              ⎥
⎢     -ⅈ⋅ω⋅t   ⎥
⎣1.0⋅ℯ        0⎦
⎡   0     0⎤
⎢          ⎥
⎢    ___   ⎥
⎣1.0⋅r₀₁  0⎦
upper right
⎡     ⅈ⋅ω⋅t⎤
⎢0  -ℯ     ⎥
⎢          ⎥
⎣0     0   ⎦
⎡0  r₀₁⎤
⎢      ⎥
⎣0   0 ⎦
lower left
⎡   0      0⎤
⎢           ⎥
⎢  -ⅈ⋅ω⋅t   ⎥
⎣-ℯ        0⎦
⎡ 0   0⎤
⎢      ⎥
⎢___   ⎥
⎣r₀₁  0⎦


In [214]:
R = sp.Matrix([[0, sp.exp(-t/(2*T1))], [sp.exp(-t/(2*T1)), 0]])
S, D = R.diagonalize()
theta = sp.symbols('theta')
1/2 * S * sp.diag(sp.exp(sp.I * theta + sp.I * sp.pi), sp.exp(sp.I * theta)) * S

Matrix([
[               0, 1.0*exp(I*theta)],
[1.0*exp(I*theta),                0]])

In [194]:
1/2 * S * sp.diag(sp.exp(-sp.I * theta - sp.I * sp.pi), sp.exp(-sp.I * theta)) * S

Matrix([
[                0, 1.0*exp(-I*theta)],
[1.0*exp(-I*theta),                 0]])